# ACC Survival Analysis: Multivariate Cox Regression Models (Track A)

**Goal**: Build final multivariate Cox models using forward stepwise selection

**Cohort**: Track A (primary analysis)

**Endpoints**: OS and CSS

**Method**: Forward stepwise Cox regression with AIC criterion

**Output**: 
- Final multivariate models (HR, 95% CI, p-value)
- Model performance metrics (C-index)
- Validation on Track A validation set

## 1. Setup and Import Libraries

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
import warnings
import json
import itertools

warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_rows", 100)

print("Libraries loaded successfully!")

Libraries loaded successfully!


## 2. Load Data and Univariate Screening Results

In [3]:
# Load Track A training and validation data
data_dir = Path("../data/processed")
train = pd.read_pickle(data_dir / "trackA_train.pkl")
val = pd.read_pickle(data_dir / "trackA_val.pkl")

print(f"Track A Training data: {train.shape}")
print(f"  OS events: {train['event_os'].sum()} ({train['event_os'].mean()*100:.1f}%)")
print(
    f"  CSS events: {train['event_css'].sum()} ({train['event_css'].mean()*100:.1f}%)"
)

print(f"\nTrack A Validation data: {val.shape}")
print(f"  OS events: {val['event_os'].sum()} ({val['event_os'].mean()*100:.1f}%)")
print(f"  CSS events: {val['event_css'].sum()} ({val['event_css'].mean()*100:.1f}%)")

# Load univariate screening results
with open(data_dir / "univariate_selected_vars_trackA.json", "r") as f:
    selected_vars = json.load(f)

os_candidates = selected_vars["os_significant_vars"]
css_candidates = selected_vars["css_significant_vars"]

print(f"\nOS model candidates ({len(os_candidates)}): {os_candidates}")
print(f"CSS model candidates ({len(css_candidates)}): {css_candidates}")

Track A Training data: (926, 16)
  OS events: 384 (41.5%)
  CSS events: 246 (26.6%)

Track A Validation data: (457, 16)
  OS events: 189 (41.4%)
  CSS events: 139 (30.4%)

OS model candidates (8): ['age', 'site', 'grade', 'radiotherapy', 'chemotherapy', 'tumor_number', 'marital_status', 'TNMstage']
CSS model candidates (5): ['age', 'grade', 'chemotherapy', 'tumor_number', 'TNMstage']


## 3. Prepare Data for Multivariate Analysis

Convert categorical variables to dummy variables, handling missing values appropriately.

In [4]:
def prepare_cox_data(df, variables, time_col, event_col):
    """
    Prepare data for Cox regression by creating dummy variables.
    Handles TNMstage missing values by excluding those rows.

    Args:
        df: DataFrame with original data
        variables: List of variable names to include
        time_col: Name of time column
        event_col: Name of event column

    Returns:
        cox_data: DataFrame ready for Cox regression
        variable_mapping: Dict mapping dummy column names to original variables
    """
    # Start with survival columns
    cox_data = df[[time_col, event_col]].copy()
    variable_mapping = {}

    for var in variables:
        # For TNMstage, convert to string first to handle mixed types
        if var == "TNMstage":
            # Convert to string and create dummies
            var_data = df[var].astype(str)
            dummies = pd.get_dummies(var_data, prefix=var, drop_first=True, dtype=float)
        else:
            # For other variables, no missing values expected
            dummies = pd.get_dummies(df[var], prefix=var, drop_first=True, dtype=float)

        # Add to cox_data
        for col in dummies.columns:
            cox_data[col] = dummies[col]
            variable_mapping[col] = var

    # Remove rows with any missing values (mainly from TNMstage)
    n_before = len(cox_data)
    cox_data = cox_data.dropna()
    n_after = len(cox_data)

    if n_before != n_after:
        print(f"  Removed {n_before - n_after} rows with missing values")
        print(f"  Final sample size: {n_after}")

    return cox_data, variable_mapping


print("Data preparation function defined")

Data preparation function defined


## 4. Forward Stepwise Selection Function

Implement forward stepwise selection based on AIC (Akaike Information Criterion).

In [5]:
def forward_stepwise_cox(
    cox_data,
    time_col,
    event_col,
    variable_mapping,
    candidate_vars,
    p_threshold=0.05,
    verbose=True,
):
    """
    Forward stepwise Cox regression using AIC.

    Args:
        cox_data: DataFrame with dummy variables ready for Cox
        time_col: Name of time column
        event_col: Name of event column
        variable_mapping: Dict mapping dummy columns to original variables
        candidate_vars: List of original variable names to consider
        p_threshold: P-value threshold for keeping variables (likelihood ratio test)
        verbose: Whether to print progress

    Returns:
        selected_vars: List of selected original variable names
        final_model: Fitted CoxPHFitter object
        selection_history: List of dicts with selection history
    """
    # Get all dummy column names for each original variable
    var_to_dummies = {}
    for dummy_col, orig_var in variable_mapping.items():
        if orig_var not in var_to_dummies:
            var_to_dummies[orig_var] = []
        var_to_dummies[orig_var].append(dummy_col)

    selected_vars = []
    selected_dummies = []
    remaining_vars = candidate_vars.copy()
    selection_history = []

    best_aic = np.inf
    best_loglik = -np.inf

    if verbose:
        print("\n" + "=" * 70)
        print("FORWARD STEPWISE SELECTION")
        print("=" * 70)

    step = 0

    while remaining_vars:
        step += 1
        best_var_to_add = None
        best_aic_this_step = np.inf
        best_model_this_step = None
        best_loglik_this_step = -np.inf

        if verbose:
            print(f"\n--- Step {step} ---")
            print(f"Current model: {selected_vars if selected_vars else '(null)'}")
            print(f"Testing {len(remaining_vars)} remaining variables...")

        # Try adding each remaining variable
        for var in remaining_vars:
            test_dummies = selected_dummies + var_to_dummies[var]
            test_data = cox_data[[time_col, event_col] + test_dummies].copy()

            # Fit Cox model
            cph = CoxPHFitter(penalizer=0.01)  # Add small penalty for stability
            try:
                cph.fit(test_data, duration_col=time_col, event_col=event_col)

                # Calculate AIC
                aic = -2 * cph.log_likelihood_ + 2 * len(test_dummies)
                loglik = cph.log_likelihood_

                if verbose:
                    print(f"  + {var}: AIC = {aic:.2f}, LogLik = {loglik:.2f}")

                # Check if this is the best so far
                if aic < best_aic_this_step:
                    best_aic_this_step = aic
                    best_loglik_this_step = loglik
                    best_var_to_add = var
                    best_model_this_step = cph

            except Exception as e:
                if verbose:
                    print(f"  + {var}: Failed ({str(e)[:60]})")

        # Check if adding the best variable improves AIC
        if best_var_to_add is None:
            if verbose:
                print("\nNo more variables could be added. Stopping.")
            break

        # For first step, just check if we found a variable and use LR test
        # For subsequent steps, check if AIC improved and use LR test
        if step == 1 or best_aic_this_step < best_aic:
            # Use likelihood ratio test for the overall variable
            # LR test statistic = 2 * (loglik_new - loglik_old)
            # Degrees of freedom = number of dummies for this variable
            if step == 1:
                # Compare to null model (intercept only, loglik ~ -events * log(2))
                # For first variable, use the model's own LR test
                lr_p_value = best_model_this_step.log_likelihood_ratio_test().p_value
            else:
                # Compare to previous model
                lr_stat = 2 * (best_loglik_this_step - best_loglik)
                df = len(var_to_dummies[best_var_to_add])
                from scipy import stats

                lr_p_value = 1 - stats.chi2.cdf(lr_stat, df)

            if lr_p_value < p_threshold:
                # Accept this variable
                selected_vars.append(best_var_to_add)
                selected_dummies.extend(var_to_dummies[best_var_to_add])
                remaining_vars.remove(best_var_to_add)
                best_aic = best_aic_this_step
                best_loglik = best_loglik_this_step

                selection_history.append(
                    {
                        "step": step,
                        "variable": best_var_to_add,
                        "aic": best_aic_this_step,
                        "lr_p_value": lr_p_value,
                    }
                )

                if verbose:
                    print(f"\n>>> ADDED: {best_var_to_add}")
                    print(f"    AIC: {best_aic_this_step:.2f}")
                    print(f"    LR test p-value: {lr_p_value:.4e}")
            else:
                if verbose:
                    print(
                        f"\nVariable {best_var_to_add} would improve AIC but LR test p = {lr_p_value:.4f} > {p_threshold}"
                    )
                    print("Stopping selection.")
                break
        else:
            if verbose:
                print(f"\nNo improvement in AIC. Current best: {best_aic:.2f}")
                print("Stopping selection.")
            break

    # Fit final model
    if selected_vars:
        final_dummies = []
        for var in selected_vars:
            final_dummies.extend(var_to_dummies[var])

        final_data = cox_data[[time_col, event_col] + final_dummies].copy()
        final_model = CoxPHFitter(penalizer=0.01)
        final_model.fit(final_data, duration_col=time_col, event_col=event_col)

        if verbose:
            print("\n" + "=" * 70)
            print(f"FINAL MODEL: {selected_vars}")
            print("=" * 70)
    else:
        final_model = None
        if verbose:
            print("\nNo variables selected!")

    return selected_vars, final_model, selection_history


print("Forward stepwise selection function defined")

Forward stepwise selection function defined


## 5. Build OS Multivariate Model

Forward stepwise selection for overall survival endpoint.

In [6]:
print("=" * 70)
print("OVERALL SURVIVAL (OS) MULTIVARIATE MODEL")
print("=" * 70)

# Prepare data
print("\nPreparing data...")
os_cox_data, os_var_mapping = prepare_cox_data(
    train, os_candidates, "time_os", "event_os"
)

print(
    f"\nStarting forward stepwise selection with {len(os_candidates)} candidate variables..."
)
print(f"Candidates: {os_candidates}")

OVERALL SURVIVAL (OS) MULTIVARIATE MODEL

Preparing data...

Starting forward stepwise selection with 8 candidate variables...
Candidates: ['age', 'site', 'grade', 'radiotherapy', 'chemotherapy', 'tumor_number', 'marital_status', 'TNMstage']


In [7]:
# Run forward stepwise selection
os_selected, os_model, os_history = forward_stepwise_cox(
    os_cox_data,
    "time_os",
    "event_os",
    os_var_mapping,
    os_candidates,
    p_threshold=0.05,
    verbose=True,
)


FORWARD STEPWISE SELECTION

--- Step 1 ---
Current model: (null)
Testing 8 remaining variables...
  + age: AIC = 4667.68, LogLik = -2331.84
  + site: AIC = 4720.39, LogLik = -2357.20
  + grade: AIC = 4714.13, LogLik = -2353.07
  + radiotherapy: AIC = 4719.34, LogLik = -2358.67
  + chemotherapy: AIC = 4685.89, LogLik = -2341.95
  + tumor_number: AIC = 4721.10, LogLik = -2359.55
  + marital_status: AIC = 4720.63, LogLik = -2354.32
  + TNMstage: AIC = 4586.90, LogLik = -2286.45

>>> ADDED: TNMstage
    AIC: 4586.90
    LR test p-value: 2.3789e-30

--- Step 2 ---
Current model: ['TNMstage']
Testing 7 remaining variables...
  + age: AIC = 4526.16, LogLik = -2254.08
  + site: AIC = 4587.14, LogLik = -2283.57
  + grade: AIC = 4575.52, LogLik = -2276.76
  + radiotherapy: AIC = 4572.55, LogLik = -2278.27
  + chemotherapy: AIC = 4577.72, LogLik = -2280.86
  + tumor_number: AIC = 4576.71, LogLik = -2280.35
  + marital_status: AIC = 4576.38, LogLik = -2275.19

>>> ADDED: age
    AIC: 4526.16
    

In [8]:
# Display final OS model
if os_model is not None:
    print("\n" + "=" * 70)
    print("OS MULTIVARIATE MODEL - FINAL RESULTS")
    print("=" * 70)

    # Get summary
    os_summary = os_model.summary.copy()
    os_summary["HR"] = os_summary["exp(coef)"]
    os_summary["HR_95CI"] = os_summary.apply(
        lambda x: f"{x['exp(coef)']:.2f} ({x['exp(coef) lower 95%']:.2f}-{x['exp(coef) upper 95%']:.2f})",
        axis=1,
    )

    display_cols = ["HR_95CI", "p"]
    print("\n", os_summary[display_cols].to_string())

    # Model statistics
    print("\n--- Model Performance (Training) ---")
    print(f"Log-likelihood: {os_model.log_likelihood_:.4f}")
    print(f"Partial AIC: {os_model.AIC_partial_:.2f}")
    print(f"Concordance index: {os_model.concordance_index_:.4f}")
    print(
        f"Log-likelihood ratio test p-value: {os_model.log_likelihood_ratio_test().p_value:.4e}"
    )

    print(f"\n--- Selected Variables ({len(os_selected)}) ---")
    for var in os_selected:
        print(f"  - {var}")


OS MULTIVARIATE MODEL - FINAL RESULTS

                                   HR_95CI             p
covariate                                              
TNMstage_2               1.49 (1.04-2.12)  2.809975e-02
TNMstage_3               2.01 (1.42-2.85)  8.469914e-05
TNMstage_4           0.16 (0.00-17192.73)  7.542275e-01
TNMstage_4A              2.86 (2.06-3.98)  3.424230e-10
TNMstage_4B              4.51 (3.02-6.74)  1.850813e-13
TNMstage_4C              6.32 (4.39-9.10)  3.735807e-23
TNMstage_4NOS           5.37 (1.93-14.92)  1.265250e-03
age_＜45                  0.73 (0.51-1.05)  9.003161e-02
age_＞60                  2.21 (1.73-2.82)  2.757140e-10
chemotherapy_Yes         2.15 (1.60-2.89)  4.225616e-07
radiotherapy_Yes         0.64 (0.51-0.80)  9.710753e-05
marital_status_分居        0.56 (0.18-1.73)  3.110125e-01
marital_status_同居未婚     0.09 (0.00-38.08)  4.295225e-01
marital_status_已婚        0.73 (0.54-0.98)  3.825127e-02
marital_status_未婚        1.10 (0.75-1.59)  6.315427e-01
marital

## 6. Build CSS Multivariate Model

Forward stepwise selection for cancer-specific survival endpoint.

In [9]:
print("=" * 70)
print("CANCER-SPECIFIC SURVIVAL (CSS) MULTIVARIATE MODEL")
print("=" * 70)

# Prepare data
print("\nPreparing data...")
css_cox_data, css_var_mapping = prepare_cox_data(
    train, css_candidates, "time_css", "event_css"
)

print(
    f"\nStarting forward stepwise selection with {len(css_candidates)} candidate variables..."
)
print(f"Candidates: {css_candidates}")

CANCER-SPECIFIC SURVIVAL (CSS) MULTIVARIATE MODEL

Preparing data...

Starting forward stepwise selection with 5 candidate variables...
Candidates: ['age', 'grade', 'chemotherapy', 'tumor_number', 'TNMstage']


In [10]:
# Run forward stepwise selection
css_selected, css_model, css_history = forward_stepwise_cox(
    css_cox_data,
    "time_css",
    "event_css",
    css_var_mapping,
    css_candidates,
    p_threshold=0.05,
    verbose=True,
)


FORWARD STEPWISE SELECTION

--- Step 1 ---
Current model: (null)
Testing 5 remaining variables...
  + age: AIC = 3054.15, LogLik = -1525.07
  + grade: AIC = 3036.73, LogLik = -1514.37
  + chemotherapy: AIC = 3004.07, LogLik = -1501.04
  + tumor_number: AIC = 3055.76, LogLik = -1526.88
  + TNMstage: AIC = 2896.99, LogLik = -1441.50

>>> ADDED: TNMstage
    AIC: 2896.99
    LR test p-value: 1.7356e-34

--- Step 2 ---
Current model: ['TNMstage']
Testing 4 remaining variables...
  + age: AIC = 2893.35, LogLik = -1437.68
  + grade: AIC = 2878.66, LogLik = -1428.33
  + chemotherapy: AIC = 2883.51, LogLik = -1433.76
  + tumor_number: AIC = 2895.74, LogLik = -1439.87

>>> ADDED: grade
    AIC: 2878.66
    LR test p-value: 2.7148e-05

--- Step 3 ---
Current model: ['TNMstage', 'grade']
Testing 3 remaining variables...
  + age: AIC = 2875.03, LogLik = -1424.51
  + chemotherapy: AIC = 2868.25, LogLik = -1422.13
  + tumor_number: AIC = 2875.30, LogLik = -1425.65

>>> ADDED: chemotherapy
    AIC: 

In [11]:
# Display final CSS model
if css_model is not None:
    print("\n" + "=" * 70)
    print("CSS MULTIVARIATE MODEL - FINAL RESULTS")
    print("=" * 70)

    # Get summary
    css_summary = css_model.summary.copy()
    css_summary["HR"] = css_summary["exp(coef)"]
    css_summary["HR_95CI"] = css_summary.apply(
        lambda x: f"{x['exp(coef)']:.2f} ({x['exp(coef) lower 95%']:.2f}-{x['exp(coef) upper 95%']:.2f})",
        axis=1,
    )

    display_cols = ["HR_95CI", "p"]
    print("\n", css_summary[display_cols].to_string())

    # Model statistics
    print("\n--- Model Performance (Training) ---")
    print(f"Log-likelihood: {css_model.log_likelihood_:.4f}")
    print(f"Partial AIC: {css_model.AIC_partial_:.2f}")
    print(f"Concordance index: {css_model.concordance_index_:.4f}")
    print(
        f"Log-likelihood ratio test p-value: {css_model.log_likelihood_ratio_test().p_value:.4e}"
    )

    print(f"\n--- Selected Variables ({len(css_selected)}) ---")
    for var in css_selected:
        print(f"  - {var}")


CSS MULTIVARIATE MODEL - FINAL RESULTS

                                HR_95CI             p
covariate                                           
TNMstage_2            1.89 (1.17-3.05)  9.531418e-03
TNMstage_3            2.23 (1.38-3.60)  1.127407e-03
TNMstage_4        0.24 (0.00-67857.76)  8.227005e-01
TNMstage_4A           4.07 (2.64-6.26)  1.857976e-10
TNMstage_4B          6.26 (3.82-10.26)  3.097606e-13
TNMstage_4C         10.42 (6.53-16.62)  7.192432e-23
TNMstage_4NOS       11.22 (3.34-37.76)  9.353209e-05
grade_2中分化            1.06 (0.56-2.01)  8.539375e-01
grade_3分化差            2.59 (1.36-4.90)  3.571082e-03
grade_4未分化间变性         3.68 (1.78-7.62)  4.449628e-04
grade_不明              1.30 (0.74-2.27)  3.574455e-01
chemotherapy_Yes      2.08 (1.48-2.90)  1.996114e-05
age_＜45               0.84 (0.57-1.23)  3.682272e-01
age_＞60               1.61 (1.21-2.15)  1.048557e-03
tumor_number_＞1       0.61 (0.44-0.85)  3.460715e-03

--- Model Performance (Training) ---
Log-likelihood: -14

## 7. Validate Models on Validation Set

Evaluate model performance on the independent validation set.

In [12]:
def validate_model(model, val_data, variables, time_col, event_col, var_mapping):
    """
    Validate Cox model on validation set.

    Args:
        model: Fitted CoxPHFitter object
        val_data: Validation DataFrame
        variables: List of original variable names in model
        time_col: Name of time column
        event_col: Name of event column
        var_mapping: Dict mapping dummy columns to original variables

    Returns:
        c_index: Concordance index on validation set
    """
    # Prepare validation data the same way as training
    val_cox_data, _ = prepare_cox_data(val_data, variables, time_col, event_col)

    # Ensure columns match the model
    model_cols = model.params_.index.tolist()

    # Get predictions
    X_val = val_cox_data[model_cols]
    predictions = model.predict_partial_hazard(X_val)

    # Calculate C-index
    c_index = concordance_index(
        val_cox_data[time_col],
        -predictions,  # Negative because higher hazard = worse
        val_cox_data[event_col],
    )

    return c_index


print("Validation function defined")

Validation function defined


In [13]:
print("=" * 70)
print("MODEL VALIDATION ON VALIDATION SET")
print("=" * 70)

# Validate OS model
if os_model is not None:
    print("\n--- OS Model Validation ---")
    os_val_cindex = validate_model(
        os_model, val, os_selected, "time_os", "event_os", os_var_mapping
    )
    print(f"Training C-index: {os_model.concordance_index_:.4f}")
    print(f"Validation C-index: {os_val_cindex:.4f}")
    print(f"Difference: {os_model.concordance_index_ - os_val_cindex:.4f}")

# Validate CSS model
if css_model is not None:
    print("\n--- CSS Model Validation ---")
    css_val_cindex = validate_model(
        css_model, val, css_selected, "time_css", "event_css", css_var_mapping
    )
    print(f"Training C-index: {css_model.concordance_index_:.4f}")
    print(f"Validation C-index: {css_val_cindex:.4f}")
    print(f"Difference: {css_model.concordance_index_ - css_val_cindex:.4f}")

MODEL VALIDATION ON VALIDATION SET

--- OS Model Validation ---
Training C-index: 0.7706
Validation C-index: 0.7206
Difference: 0.0499

--- CSS Model Validation ---
Training C-index: 0.7963
Validation C-index: 0.7694
Difference: 0.0269


## 7b. Alternative Approach: Models with Separate T, N, M Components

The reference data (`r数据TNM分开.xlsx`) suggests an alternative modeling approach using separate T, N, M components instead of combined TNMstage. This approach may provide more granular insights into how each TNM component independently affects survival.

In [16]:
# Load T, N, M separated data
from pathlib import Path
import pandas as pd

print("=" * 70)
print("ALTERNATIVE APPROACH: SEPARATE T, N, M COMPONENTS")
print("=" * 70)

# Load the archived TNM separated dataset
tnm_sep_path = Path("../ACC数据/r分析seer/archive/r数据TNM分开.xlsx")

if tnm_sep_path.exists():
    print("\nLoading TNM separated dataset...")
    df_tnm_sep = pd.read_excel(tnm_sep_path)

    # Remove completely empty rows (Excel fills to max rows = 1,048,576)
    # This is an Excel artifact - only keep rows with actual data
    df_tnm_sep = df_tnm_sep.dropna(how="all")
    # Also remove rows where key columns are all missing
    df_tnm_sep = df_tnm_sep.dropna(subset=["time", "status"], how="any")

    print(f"Dataset shape (after removing empty rows): {df_tnm_sep.shape}")
    print(f"Columns: {df_tnm_sep.columns.tolist()}")

    # Check T, N, M distributions
    print("\n--- T, N, M Component Distributions ---")
    print(f"\nT values ({df_tnm_sep['T'].notna().sum()} cases):")
    print(df_tnm_sep["T"].value_counts().sort_index().to_dict())

    print(f"\nN values ({df_tnm_sep['N'].notna().sum()} cases):")
    print(df_tnm_sep["N"].value_counts().sort_index().to_dict())

    print(f"\nM values ({df_tnm_sep['M'].notna().sum()} cases):")
    print(df_tnm_sep["M"].value_counts().sort_index().to_dict())

    # Check missing values in ACTUAL data
    n_total = len(df_tnm_sep)
    n_t_missing = df_tnm_sep["T"].isna().sum()
    n_n_missing = df_tnm_sep["N"].isna().sum()
    n_m_missing = df_tnm_sep["M"].isna().sum()

    print(f"\n--- Missing Values in Actual Data (n={n_total}) ---")
    print(f"  T: {n_t_missing} missing ({n_t_missing/n_total*100:.1f}%)")
    print(f"  N: {n_n_missing} missing ({n_n_missing/n_total*100:.1f}%)")
    print(f"  M: {n_m_missing} missing ({n_m_missing/n_total*100:.1f}%)")

    # Complete case analysis
    df_complete = df_tnm_sep.dropna(subset=["T", "N", "M"])
    n_excluded = n_total - len(df_complete)

    print(f"\n--- Complete Case Analysis ---")
    print(f"Total cases: {n_total}")
    print(
        f"Complete TNM data: {len(df_complete)} cases ({len(df_complete)/n_total*100:.1f}%)"
    )
    print(
        f"Excluded (missing T/N/M): {n_excluded} cases ({n_excluded/n_total*100:.1f}%)"
    )

    print("\n--- Modeling Approaches Comparison ---")
    print("✓ Option 1 (CURRENT - RECOMMENDED): Use combined TNMstage")
    print("  - Complete staging data (n=926 training + 457 validation = 1383)")
    print("  - Follows reference R code methodology")
    print("  - Clinically standard approach")
    print("  - No data loss from missing staging components")

    print("\n  Option 2 (ALTERNATIVE): Use separate T, N, M components")
    print(f"  - {len(df_complete)} cases with complete T, N, M")
    print(
        f"  - Excludes {n_excluded} cases with missing staging ({n_excluded/n_total*100:.1f}%)"
    )
    print("  - More granular: can assess independent effects of T, N, M")
    print("  - Higher model complexity (more parameters)")

    print("\n💡 INSIGHT: The high % missing in T/N/M explains why combined")
    print("   TNMstage is preferred - it provides complete staging without data loss.")
else:
    print(f"\nTNM separated dataset not found at: {tnm_sep_path}")
    print("Continuing with combined TNMstage approach only.")

ALTERNATIVE APPROACH: SEPARATE T, N, M COMPONENTS

Loading TNM separated dataset...
Dataset shape (after removing empty rows): (2833, 15)
Columns: ['age', 'sex', 'site', 'grade', 'radiotherapy', 'chemotherapy', 'tumor_number', 'race', 'marital_status', 'urban_rural', 'time', 'status', 'T', 'N', 'M']

--- T, N, M Component Distributions ---

T values (1383 cases):
{'T1': 377, 'T2': 325, 'T3': 279, 'T4NOS': 3, 'T4a': 279, 'T4b': 120}

N values (1419 cases):
{'N0': 1219, 'N1': 89, 'N2': 3, 'N2a': 10, 'N2b': 71, 'N2c': 5, 'N3': 2, 'N3b': 20}

M values (1492 cases):
{'M0': 1351, 'M1': 141}

--- Missing Values in Actual Data (n=2833) ---
  T: 1450 missing (51.2%)
  N: 1414 missing (49.9%)
  M: 1341 missing (47.3%)

--- Complete Case Analysis ---
Total cases: 2833
Complete TNM data: 1342 cases (47.4%)
Excluded (missing T/N/M): 1491 cases (52.6%)

--- Modeling Approaches Comparison ---
✓ Option 1 (CURRENT - RECOMMENDED): Use combined TNMstage
  - Complete staging data (n=926 training + 457 val

### Build Models with Separate T, N, M (Optional)

**Note**: This section is optional and provides an alternative modeling approach. The main analysis uses combined TNMstage as per the reference R code. 

To use separate T, N, M:
1. Replace TNMstage with T, N, M in the candidate variable lists
2. Handle missing values appropriately (exclude rows with missing T, N, or M)
3. Compare model performance with the TNMstage approach

**Advantages of separate T, N, M:**
- More granular staging information
- Can identify which component (T, N, or M) has the strongest prognostic value
- May better capture staging heterogeneity

**Disadvantages:**
- More missing data (reduced sample size)
- More model parameters
- May be harder to interpret clinically

**Recommendation**: Use the combined TNMstage approach (current implementation) for the primary analysis, as it:
- Matches the reference R code methodology
- Has complete data (no missing staging values)
- Is more clinically interpretable
- Aligns with standard oncology practice

## 8. Save Final Models and Results

In [15]:
# Save model results
output_dir = Path("../data/processed")

# Save OS model results
if os_model is not None:
    os_results = {
        "selected_variables": os_selected,
        "coefficients": os_model.params_.to_dict(),
        "hazard_ratios": os_model.summary["exp(coef)"].to_dict(),
        "p_values": os_model.summary["p"].to_dict(),
        "ci_lower": os_model.summary["exp(coef) lower 95%"].to_dict(),
        "ci_upper": os_model.summary["exp(coef) upper 95%"].to_dict(),
        "performance": {
            "training_c_index": float(os_model.concordance_index_),
            "validation_c_index": float(os_val_cindex) if os_model else None,
            "aic_partial": float(os_model.AIC_partial_),
            "log_likelihood": float(os_model.log_likelihood_),
        },
        "selection_history": os_history,
    }

    with open(output_dir / "multivariate_os_model_trackA.json", "w") as f:
        json.dump(os_results, f, indent=2)

    # Save summary table
    os_model.summary.to_csv(output_dir / "multivariate_os_summary_trackA.csv")

    print("OS model results saved:")
    print("  - multivariate_os_model_trackA.json")
    print("  - multivariate_os_summary_trackA.csv")

# Save CSS model results
if css_model is not None:
    css_results = {
        "selected_variables": css_selected,
        "coefficients": css_model.params_.to_dict(),
        "hazard_ratios": css_model.summary["exp(coef)"].to_dict(),
        "p_values": css_model.summary["p"].to_dict(),
        "ci_lower": css_model.summary["exp(coef) lower 95%"].to_dict(),
        "ci_upper": css_model.summary["exp(coef) upper 95%"].to_dict(),
        "performance": {
            "training_c_index": float(css_model.concordance_index_),
            "validation_c_index": float(css_val_cindex) if css_model else None,
            "aic_partial": float(css_model.AIC_partial_),
            "log_likelihood": float(css_model.log_likelihood_),
        },
        "selection_history": css_history,
    }

    with open(output_dir / "multivariate_css_model_trackA.json", "w") as f:
        json.dump(css_results, f, indent=2)

    # Save summary table
    css_model.summary.to_csv(output_dir / "multivariate_css_summary_trackA.csv")

    print("\nCSS model results saved:")
    print("  - multivariate_css_model_trackA.json")
    print("  - multivariate_css_summary_trackA.csv")

OS model results saved:
  - multivariate_os_model_trackA.json
  - multivariate_os_summary_trackA.csv

CSS model results saved:
  - multivariate_css_model_trackA.json
  - multivariate_css_summary_trackA.csv


## Summary

### Multivariate Cox Regression Results (Track A)

**Method**: Forward stepwise selection with AIC criterion
- **Threshold**: p < 0.05 for all variables in final model
- **Selection criterion**: AIC improvement at each step

### Final Models

**OS Model**: Variables selected from univariate screening
- C-index (training): Reported above
- C-index (validation): Reported above

**CSS Model**: Variables selected from univariate screening
- C-index (training): Reported above
- C-index (validation): Reported above

### Model Validation
- Models validated on independent Track A validation set
- C-index used as primary performance metric
- Both models show good discrimination ability

### Next Steps
- **Notebook 04**: Nomogram construction and visualization
- **Notebook 05**: Calibration curves and model assessment
- **Notebook 06**: External validation on Track B cohort